In [31]:
import os
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
wd = os.getcwd()
from constants import SAMPLES
print (f'working directory: {wd}')


bucket = 'fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01'

working directory: /Users/whuan/dev/ist_benchmarking


# Install pacage and grab scripts (only need to run once)
You won't need to do it again for other notebooks.

In [8]:
# # # copy scripts
# cmd = f'gsutil cp gs://{bucket}/notebooks/* {wd}'
# print (cmd)
# ! {cmd}

# # install pacakges
! pip install squidpy==1.2.3 
! pip install geopandas==0.13.0 
! pip install -U kaleido
! pip install rtree==1.0.1
! pip install rasterio
! pip install plotly
! pip install nbformat==5.10.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Copy data from bucket to VM

In [3]:
# # copy data
bucket = 'fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01'
cmd = f'gsutil cp -r gs://{bucket}/data {wd}'
print (cmd)
! {cmd}

gsutil cp -r gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data /home/jupyter/iST_Platform_Comparison/edit
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/Sample_Info_HTMA_TMA - Sheet1.csv...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/Sample_Info_NORMAL_TMA - Sheet1.csv...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/cell_by_gene_csv/cosmx_multitissue_htma_cell_by_gene.parquet.gzip...
- [3 files][ 27.0 MiB/ 27.0 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/cell_by_gene_csv/cosmx_multitissue_normal_cell_by_gene.parquet.gzip...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/cell_by_gene_csv/merscope_breast_htma_cell_by_gene.parq

Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/cell_by_gene.csv...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/cell_metadata.csv...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/dapi_by_core/mosaic_DAPI_z3.tif_.gstmp...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/merscope_breast_normal_polygon.cpg...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/merscope_breast_normal_polygon.dbf...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/merscope_breast_normal_polygon.prj...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/merscope_breast_normal_polygon.shp...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_normal/merscope_breast_normal_polygon.shx...
Copying gs://fc-809bcaf6-66cc-4edc-a6ad-f8c671d31a01/data/merscope_breast_norma

In [2]:
sample = '2024_cosmx_multitissue_htma'

# Cell level data

In [3]:
gdf = gpd.read_parquet(f'{wd}/data/cell_level_csv/{sample}_cell_level.parquet.gzip')
gdf.head()

# you can load the data faster use pandas with pyarrow engine if you don't care the geometry column
# df = pd.read_parquet(f'data/cell_by_gene_csv/{sample}_cell_by_gene.parquet.gzip', engine='pyarrow')


,orig.ident,transcript_counts,nFeature_RNA,nCount_negprobes,nFeature_negprobes,fov,Area,AspectRatio,x_FOV_px,y_FOV_px,...,qcFlagsFOV,i.median_negprobes,i.negprobes_quantile_0.9,i.median_RNA,i.RNA_quantile_0.9,x_centroid,y_centroid,core,tissue_type,geometry
0,c,330.0,90.0,0.0,0.0,100,5551,0.89,581,44,...,Pass,29.0,36.4,46.0,165.1,5.861764,4.214475,64,HNSCC,POINT (4.21448 5.86176)
1,c,233.0,91.0,1.0,1.0,100,8718,1.19,2066,48,...,Pass,29.0,36.4,46.0,165.1,5.861283,4.393092,64,HNSCC,POINT (4.39309 5.86128)
2,c,691.0,163.0,1.0,1.0,100,8974,0.88,1278,645,...,Pass,29.0,36.4,46.0,165.1,5.789475,4.298311,64,HNSCC,POINT (4.29831 5.78947)
3,c,155.0,98.0,0.0,0.0,100,5396,0.71,2987,3245,...,Pass,29.0,36.4,46.0,165.1,5.476744,4.503871,64,HNSCC,POINT (4.50387 5.47674)
4,c,13.0,12.0,0.0,0.0,100,3383,0.80,3286,3238,...,Pass,29.0,36.4,46.0,165.1,5.477586,4.539835,64,HNSCC,POINT (4.53984 5.47759)


In [5]:
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot('tissue_type',aspect=1, markersize=1, ax=ax, legend=True, cmap='tab20c')
plt.show()

# Cell by gene data

In [47]:
sample = SAMPLES[-1]
print (sample)

df = pd.read_parquet(f'{wd}/data/cell_by_gene_csv/{sample}_cell_by_gene.parquet.gzip', engine='pyarrow')
df = df.reset_index(drop=True)
df.head()

2024_cosmx_multitissue_tumor2


,core,tissue_type,AATK,ABL1,ABL2,ACACB,ACE,ACKR1,ACKR3,ACKR4,...,WNT7B,WNT9A,XBP1,XCL1/2,XKR4,YBX3,YES1,ZBTB16,ZFP36,cell_id
0,30,LN B-cell lymphoma,1,6,8,6,18,11,6,11,...,11,7,39,2,4,54,1,12,24,c_2_100_0
1,30,LN B-cell lymphoma,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,c_2_100_1
2,30,LN B-cell lymphoma,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,c_2_100_10
3,30,LN B-cell lymphoma,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,c_2_100_100
4,30,LN B-cell lymphoma,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,c_2_100_1000


In [48]:
import anndata as ad

# # for sample in SAMPLES[-2:-1]:
# df = pd.read_parquet(f'{wd}/data/cell_by_gene_csv/{sample}_cell_by_gene.parquet.gzip', engine='pyarrow')
# df = df.reset_index(drop=True)
numeric_cols = df.select_dtypes(include=['number']).columns
non_numeric_cols = df.select_dtypes(exclude=['number']).columns

df_numeric = df[numeric_cols].copy()      
df_non_numeric = df[non_numeric_cols].copy()

adata = ad.AnnData(X=df_numeric.values)
adata.obs_names = df_numeric.index
adata.var_names = df_numeric.columns
adata.var['gene'] = adata.var.index
adata.obs = df_non_numeric
adata.write_h5ad(f'{wd}/data/cell_by_gene_csv/{sample}_cell_by_gene.h5ad')
            

In [49]:
print (adata)
print (adata.n_vars)
print (adata.var.head(5))
print (adata.obs.head(5))

AnnData object with n_obs × n_vars = 253345 × 1207
    obs: 'core', 'tissue_type', 'cell_id'
    var: 'gene'
1207
        gene
AATK    AATK
ABL1    ABL1
ABL2    ABL2
ACACB  ACACB
ACE      ACE
  core         tissue_type       cell_id
0   30  LN B-cell lymphoma     c_2_100_0
1   30  LN B-cell lymphoma     c_2_100_1
2   30  LN B-cell lymphoma    c_2_100_10
3   30  LN B-cell lymphoma   c_2_100_100
4   30  LN B-cell lymphoma  c_2_100_1000


In [50]:
print (adata.obs.head(5))


  core         tissue_type       cell_id
0   30  LN B-cell lymphoma     c_2_100_0
1   30  LN B-cell lymphoma     c_2_100_1
2   30  LN B-cell lymphoma    c_2_100_10
3   30  LN B-cell lymphoma   c_2_100_100
4   30  LN B-cell lymphoma  c_2_100_1000


# Gene level data

In [11]:
df = pd.read_csv(f'{wd}/data/gene_level_csv/gene_level_csv_{sample}.csv', engine='pyarrow')
df.head()

,core,gene,tissue_type,count,code_type
0,1,ABCC11,Bladder,90,gene
1,1,ACTA2,Bladder,935,gene
2,1,ACTG2,Bladder,802,gene
3,1,ADAM9,Bladder,1019,gene
4,1,ADGRE5,Bladder,271,gene


# Transcript level data

In [12]:
gdf = gpd.read_parquet(f'{wd}/data/transcript_level_csv/{sample}_transcript_level.parquet.gzip')
gdf.head()

,core,tissue_type,transcript_id,cell_id,overlaps_nucleus,gene,global_x,global_y,z_location,qv,fov_name,nucleus_distance,codeword_index,modality,geometry
0,170,Lymph node,281981782851630,fbamikmj-1,0,NKG7,153.443268,1999.059326,26.074429,40.000000,G1,0.816286,334,xenium,POINT (1999.05933 153.44327)
1,170,Lymph node,281981782851643,fbamikmj-1,1,ITGAX,156.906540,1995.951904,22.569859,40.000000,G1,0.000000,195,xenium,POINT (1995.95190 156.90654)
2,170,Lymph node,281981782851650,fbamikmj-1,1,ITGAX,157.524414,1993.435303,25.381569,31.106482,G1,0.000000,195,xenium,POINT (1993.43530 157.52441)
3,170,Lymph node,281981782851652,gnmpgfpd-1,1,LPXN,158.078476,1977.405029,27.601223,40.000000,G1,0.000000,316,xenium,POINT (1977.40503 158.07848)
4,170,Lymph node,281981782851656,fbamdhdg-1,0,TCIM,159.661682,1983.071289,22.600060,40.000000,G1,0.426842,140,xenium,POINT (1983.07129 159.66168)
